In [ ]:
### cannot be done in test_ben0 conda environment due to collisions. Instead, I'll make another conda env
### specifically for data viz. I will call it: data_viz0 (lol)
### remember, use test_ben0 for actual inference.

import numpy as np
import mdtraj as md
import nglview as nv

In [ ]:
import numpy as np
import mdtraj as md
import nglview as nv

# Parameters
scaling_factor = 30
pdb_path = "/home/bfd21/rds/hpc-work/sample_cyclic_md/ligand-only/dummy1/l1.pdb"
npz_files = [
    "/home/bfd21/rds/hpc-work/tbg/result_data/Dec-3-2024/bb_all_sc_adj_batch-0.npz",
    "/home/bfd21/rds/hpc-work/tbg/result_data/Dec-3-2024/bb_all_sc_adj_batch-1.npz",
]

# Load topology and ensure bonds are loaded
pdb = md.load(pdb_path)
topology = pdb.topology

# Load and process generated samples
all_samples = []
for npz_file in npz_files:
    data = np.load(npz_file)
    samples_np = data["samples_np"]
    # Reshape samples to match the number of particles and scale them
    n_particles = len(list(topology.atoms))
    samples = samples_np.reshape(-1, n_particles, 3) * scaling_factor
    all_samples.append(samples)

# Combine all samples into a single array
all_samples = np.concatenate(all_samples, axis=0)

# Create an MDTraj trajectory with the combined samples
traj = md.Trajectory(
    xyz=all_samples,  # Shape: (n_frames, n_atoms, 3)
    topology=topology
)

# Center the trajectory around the origin
traj.center_coordinates()

# Visualize using nglview
view = nv.show_mdtraj(traj)

# Use "cartoon" for overall structure and "ball+stick" for bonds
view.add_representation("cartoon", selection="protein")  # Highlights protein backbone
view.add_representation("ball+stick", selection="all", radius=0.5)  # Ball-and-stick for all atoms

# Ensure all bonds in the PDB topology are displayed
for bond in topology.bonds:
    atom1_idx = bond[0].index
    atom2_idx = bond[1].index
    view.add_representation(
        "licorice", 
        selection=f"@{atom1_idx},{atom2_idx}", 
        radius=0.3, 
        bondScale=1.2
    )

# Display the visualization
view


NGLWidget(max_frame=134)

In [ ]:
### NOW ADD TRAINING DATA VIZUALIZATION